# Loopy: Controlling data layout

## Setup code

In [1]:
import numpy as np
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
import loopy as lp

from loopy.version import LOOPY_USE_LANGUAGE_VERSION_2018_2

In [3]:
ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx)

Choose platform:
[0] <pyopencl.Platform 'Portable Computing Language' at 0x7fd0ffe3c6e8>
[1] <pyopencl.Platform 'Intel(R) OpenCL' at 0x33094e8>


Choice [0]: 0


Set the environment variable PYOPENCL_CTX='0' to avoid being asked again.


## A kernel on a structured array

In [4]:
knl = lp.make_kernel(
    "{[el,dof, comp]: "
        "0<=el<nels "
        "and 0<=dof<14 "
        "and 0<=comp < 3}",
    "D[el, dof, comp] = eps[el] * E[el, dof, comp]")

knl = lp.set_options(knl, write_cl=True)

In [5]:
eps = np.random.randn(500)
E = cl.clrandom.rand(queue, (500, 14, 3), dtype=np.float64)

In [6]:
mknl = knl.copy()
evt, _ = mknl(queue, eps=eps, E=E)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))
#if __OPENCL_C_VERSION__ < 120
#pragma OPENCL EXTENSION cl_khr_fp64: enable
#endif

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global double *__restrict__ D, __global double const *__restrict__ E, __global double const *__restrict__ eps, int const nels)
{
  for (int el = 0; el <= -1 + nels; ++el)
    for (int dof = 0; dof <= 13; ++dof)
      for (int comp = 0; comp <= 2; ++comp)
        D[42 * el + 3 * dof + comp] = eps[el] * E[42 * el + 3 * dof + comp];
}



## Changing the layout

`E` and `D` are currently laid out as AoS. What if I want SoA?

In [8]:
mknl = knl

mknl  = lp.tag_array_axes(mknl, "E", "c,c,sep")
mknl = lp.tag_inames(mknl, {"comp": "unr"})
mknl = lp.prioritize_loops(mknl, "el,dof,comp")

# change data format of E
copy_knl = lp.make_copy_kernel("c,c,sep")
copy_knl = lp.fix_parameters(copy_knl, n2=3)
evt, E_new = copy_knl(queue, input=E)

evt, _ = mknl(queue, eps=eps, E=E_new)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))
#if __OPENCL_C_VERSION__ < 120
#pragma OPENCL EXTENSION cl_khr_fp64: enable
#endif

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global double *__restrict__ D, __global double const *__restrict__ E_s0, __global double const *__restrict__ E_s1, __global double const *__restrict__ E_s2, __global double const *__restrict__ eps, int const nels)
{
  for (int el = 0; el <= -1 + nels; ++el)
    for (int dof = 0; dof <= 13; ++dof)
    {
      D[42 * el + 3 * dof] = eps[el] * E_s0[14 * el + dof];
      D[1 + 42 * el + 3 * dof] = eps[el] * E_s1[14 * el + dof];
      D[2 + 42 * el + 3 * dof] = eps[el] * E_s2[14 * el + dof];
    }
}



May want to add padding (demo).

---

Grouped padding exists as well.